# Load data in pandas

In [0]:
test_data_a = "https://drive.google.com/uc?id=1rbhlUfffty5pHWPMPKBhR76dz4jp7ZFb&export=download"

train_data_a = "https://drive.google.com/uc?id=1UU5wVku8v79aEKknkOBMrKNDZrTxUpJZ&export=download"

test_data = "https://drive.google.com/uc?id=1Yabjq972kr7pA-HST7MP-9gR0xTSOjs_&export=download"

train_data = "https://drive.google.com/uc?id=1SButl51QPhJ0WbFniXFLFO-YoxzJ0V3p&export=download"

valid_data = "https://drive.google.com/uc?id=17EONLP166qnOeedOZrIl5xu5zznRH9b3&export=download"

valid_data_a = "https://drive.google.com/uc?id=11Txw4zCcyYCJ6TCsr90F-LF2_b63HiZ2&export=download"

In [2]:
!pip install pandas==0.24.1 # Google colab pandas is too old to read these pickle
!pip install scipy==1.2.1 # Scipy is too low to have the function

In [0]:
def download(url):
  import requests
  from io import BytesIO
  r = requests.get(url)
  data = r.content
  df = pd.read_pickle(BytesIO(data),compression = None)
  return df

In [0]:
import pandas as pd


test_data = download(test_data)
test_data_a = download(test_data_a)
train_data = download(train_data)
train_data_a = download(train_data_a)
valid_data = download(valid_data)
valid_data_a = download(valid_data_a)

In [5]:
train_data.columns

Index(['ID', 'Text', 'Pronoun', 'Pronoun-offset', 'A', 'A-offset', 'A-coref',
       'B', 'B-offset', 'B-coref', 'URL', 'bert_tokens_torch', 'name_list',
       'token_map_bert', 'sentence_map_bert', 'A_dist_bert', 'B_dist_bert',
       'A_pos_bert', 'B_pos_bert', 'pron_pos_bert', 'A_idx_bert_torch',
       'B_idx_bert_torch', 'pron_idx_bert_torch', 'topic', 'topic_A',
       'topic_B', 'name_idx_bert_torch', 'neither_list',
       'neither_idx_bert_torch', 'label'],
      dtype='object')

# Fine-tune BERT-large and mlp model

In [6]:
!pip install pytorch-pretrained-bert
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch.nn as nn

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
# form train_data
train_all = pd.concat([train_data, valid_data], ignore_index=True, sort=False)
# train_all = train_data
train_all['indexed_tokens'] = train_all['bert_tokens_torch'].apply(lambda x: tokenizer.convert_tokens_to_ids(x))
train_indexed_tokens = list(train_all['indexed_tokens'])
train_offsets_A = list(train_all['A_idx_bert_torch'])
train_offsets_B = list(train_all['B_idx_bert_torch'])
train_offsets_pron = list(train_all['pron_idx_bert_torch'])
train_offsets_other = list(train_all['neither_idx_bert_torch'])
train_offsets_cls = [np.array([0])]*len(train_offsets_other)
train_labels = list(train_all['label'])
print('Number of training data is:',len(train_all))


# form test_data
test_data['indexed_tokens'] = test_data['bert_tokens_torch'].apply(lambda x: tokenizer.convert_tokens_to_ids(x))
test_indexed_tokens = list(test_data['indexed_tokens'])
test_offsets_A = list(test_data['A_idx_bert_torch'])
test_offsets_B = list(test_data['B_idx_bert_torch'])
test_offsets_pron = list(test_data['pron_idx_bert_torch'])
test_offsets_other = list(test_data['neither_idx_bert_torch'])
test_offsets_cls = [np.array([0])]*len(test_offsets_other)
test_labels = list(test_data['label'])
print('Number of testing data is:',len(test_data))

Number of training data is: 2454
Number of testing data is: 2000


In [0]:
import numpy as np

torch_rd = 19415
np.random.seed(seed=0)

import torch
torch.manual_seed(torch_rd)

model_name = "bert_large_rd"+str(torch_rd)

CV = 0

In [9]:
max([len(i) for i in test_indexed_tokens])

348

In [0]:
from sklearn.model_selection import KFold 
kf = KFold(n_splits=5) 
k = 0
for train_index, test_index in kf.split(train_labels):
  if k == CV:
    break
  k = k + 1

In [11]:
test_index

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [0]:
if CV in [0,1,2,3,4]:
  train_indexed_tokens,test_indexed_tokens = [train_indexed_tokens[i] for i in list(train_index)],[train_indexed_tokens[i] for i in list(test_index)]
  train_offsets_A,test_offsets_A = [train_offsets_A[i] for i in list(train_index)],[train_offsets_A[i] for i in list(test_index)]
  train_offsets_B,test_offsets_B = [train_offsets_B[i] for i in list(train_index)],[train_offsets_B[i] for i in list(test_index)]
  train_offsets_pron,test_offsets_pron = [train_offsets_pron[i] for i in list(train_index)],[train_offsets_pron[i] for i in list(test_index)]
  train_offsets_other,test_offsets_other = [train_offsets_other[i] for i in list(train_index)],[train_offsets_other[i] for i in list(test_index)]
  train_offsets_cls,test_offsets_cls = [train_offsets_cls[i] for i in list(train_index)],[train_offsets_cls[i] for i in list(test_index)]
  train_labels,test_labels = [train_labels[i] for i in list(train_index)],[train_labels[i] for i in list(test_index)]
  
if CV == 6:
  train_indexed_tokens = train_indexed_tokens + test_indexed_tokens
  train_offsets_A = train_offsets_A + test_offsets_A
  train_offsets_B = train_offsets_B + test_offsets_B
  train_offsets_pron = train_offsets_pron + test_offsets_pron
  train_offsets_other = train_offsets_other + test_offsets_other
  train_offsets_cls = train_offsets_cls + test_offsets_cls
  train_labels = train_labels + test_labels
  

In [0]:
class MLP(nn.Module):
    def __init__(self, bert_hidden_size):
        super().__init__()
        self.bert_hidden_size = bert_hidden_size
        self.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(bert_hidden_size, 1),

        )
        
    def forward(self, vector):
        return self.fc(vector)

In [0]:
def change_dropout(module,p):
    for name in module.__dict__['_modules']:
        if name == "dropout":
            module.__dict__['_modules'][name].p = p
        else:
            change_dropout(module.__dict__['_modules'][name],p)

In [0]:
def extract_vectors(bert_last_layer, offsets, offsets_mask):
    extracted_vectors = bert_last_layer.gather(
        1, offsets.unsqueeze(2).expand(-1, -1, 1))
    vector_masked = extracted_vectors.mul(offsets_mask.unsqueeze(2).float()).sum(1)/(offsets_mask.sum(1).unsqueeze(1).float())
    return vector_masked
  
def get_padded_and_mask(sequence, left, right):
    seq_ori = sequence[left:right]
    seq_padded = pad_sequence([torch.Tensor(v) for v in seq_ori],batch_first = True).cuda().long()
    mask = (seq_padded != 0).cuda().long()
    return seq_padded, mask

In [16]:
from torch.nn.utils.rnn import pad_sequence
sub_batch = 2 # batch size越大越差？不知道什么操作
acc_time = 8
acc_counter = 0
EPOCHS = 1
  

loss_c = nn.CrossEntropyLoss()
loss_c.cuda()
bert = BertModel.from_pretrained('bert-large-uncased')
bert =  bert.cuda()
change_dropout(bert,0.15) # 上面28的情况下改成dropout 0.1会爆内存

mlp = MLP(1024).cuda()
change_dropout(mlp,0)

#opt = torch.optim.Adam([
#     {'params': bert.parameters(), 'lr': 2e-5},{'params': mlp.parameters()}
# ], lr=1e-3)
# opt = torch.optim.SGD(list(bert.parameters()) + list(mlp.parameters()), lr = 2e-5, momentum = 2e-4)
opt = torch.optim.Adam(list(bert.parameters()) + list(mlp.parameters()), lr=2e-5)

for e in range(1, EPOCHS+1):
  for b in range(0,len(train_offsets_cls),sub_batch):
    start = b
    end = min(len(train_offsets_cls),b+sub_batch)
  
    tr_X_padded, attention_mask = get_padded_and_mask(train_indexed_tokens, start, end)
    tr_Y = torch.tensor(train_labels[start:end]).cuda()
    tr_A_padded, tr_A_mask = get_padded_and_mask(train_offsets_A, start, end)
    tr_B_padded, tr_B_mask = get_padded_and_mask(train_offsets_B, start, end)
    tr_other_padded, tr_other_mask = get_padded_and_mask([np.array(list(i)+list(j)) for i,j in zip(train_offsets_cls,train_offsets_pron)], start, end)
    segments_tensors = torch.zeros(tr_X_padded.size()).cuda().long()
    
    encoded_bert, _ = bert(tr_X_padded, segments_tensors,attention_mask,output_all_encoded_layers=False)
    
    pred_ = mlp(encoded_bert)
    pred = torch.cat([extract_vectors(pred_, tr_A_padded, tr_A_mask),\
                              extract_vectors(pred_, tr_B_padded, tr_B_mask),extract_vectors(pred_, tr_other_padded, tr_other_mask)],1)

    loss = loss_c(pred, tr_Y)

    loss.backward()
    acc_counter = acc_counter+1
    if acc_counter == acc_time:
      opt.step()
      acc_counter = 0
      opt.zero_grad()
      torch.cuda.empty_cache()
    if b%100 == 0:
      print ("Epoch =",e,"loss = ",loss.item())

Epoch = 1 loss =  1.1574223041534424
Epoch = 1 loss =  1.0246069431304932
Epoch = 1 loss =  0.8755170106887817
Epoch = 1 loss =  0.6332923173904419
Epoch = 1 loss =  1.5022794008255005
Epoch = 1 loss =  0.22496867179870605
Epoch = 1 loss =  0.06795275211334229
Epoch = 1 loss =  0.7569471597671509
Epoch = 1 loss =  1.3980152606964111
Epoch = 1 loss =  0.5227921605110168
Epoch = 1 loss =  0.20769333839416504
Epoch = 1 loss =  0.17039060592651367
Epoch = 1 loss =  0.23409408330917358
Epoch = 1 loss =  0.5435210466384888
Epoch = 1 loss =  0.14981615543365479
Epoch = 1 loss =  0.04577898979187012
Epoch = 1 loss =  0.4594902992248535
Epoch = 1 loss =  0.014090299606323242
Epoch = 1 loss =  0.44399213790893555
Epoch = 1 loss =  0.0655813217163086


In [17]:
re = []
bert.eval()
mlp.eval()
with torch.no_grad():
  for b in range(0,len(test_offsets_cls),1):
    start = b
    end = min(len(test_offsets_cls),b+1)
    tr_X_padded, attention_mask = get_padded_and_mask(test_indexed_tokens, start, end)
    tr_Y = torch.tensor(test_labels[start:end]).cuda()
    tr_A_padded, tr_A_mask = get_padded_and_mask(test_offsets_A, start, end)
    tr_B_padded, tr_B_mask = get_padded_and_mask(test_offsets_B, start, end)
    tr_other_padded, tr_other_mask = get_padded_and_mask([np.array(list(i)+list(j)) for i,j in zip(test_offsets_cls,test_offsets_pron)], start, end)
    segments_tensors = torch.zeros(tr_X_padded.size()).cuda().long()
    
    encoded_bert, _ = bert(tr_X_padded, segments_tensors,attention_mask,output_all_encoded_layers=False)
    
    pred_ = mlp(encoded_bert)
    pred = torch.cat([extract_vectors(pred_, tr_A_padded, tr_A_mask),\
                              extract_vectors(pred_, tr_B_padded, tr_B_mask),extract_vectors(pred_, tr_other_padded, tr_other_mask)],1)
    pred_data = pred.data.cpu().numpy()
    re.append(pred_data)

loss_c = nn.CrossEntropyLoss()

loss_data = loss_c(torch.Tensor(np.concatenate(re).reshape(-1,3)),torch.Tensor(test_labels).long())
print('Loss for test data is:', loss_data)

Loss for test data is: tensor(0.4078)


In [0]:
import os
from joblib import dump, load
from scipy.special import softmax
pred = np.zeros((len(train_all),3))

if CV in [0,1,2,3,4]:
  if os.path.exists(model_name+'+basic.joblib'):
    pred = load(model_name+'+basic.joblib')
    os.remove(model_name+'+basic.joblib')
  pred[test_index,:] = softmax(np.concatenate(re).reshape(-1,3),axis = 1)
  dump(pred,model_name+'+basic.joblib')

if CV == 5:
  if os.path.exists(model_name+'+basic.joblib'):
    pred_tr = load(model_name+'+basic.joblib')
    os.remove(model_name+'+basic.joblib')
  pred = softmax(np.concatenate(re).reshape(-1,3),axis = 1)
  dump((pred_tr,pred),model_name+'+basic.joblib')
  
if CV == 6:
  dump((bert,mlp),model_name +'.joblib')
  


  
  

      
      